## New Data Predictions 
On Tree-based models

In [80]:
# Libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_date

# Set float display to avoid scientific notation
pd.set_option('display.float_format', '{:.2f}'.format)

# Wranings 
import warnings
warnings.filterwarnings('ignore')

StatementMeta(, 3ecf38ca-948d-4fd9-99ab-6cbf91e300e1, 82, Finished, Available, Finished)

### Read Data

In [81]:
# Read data 
new_df = pd.read_parquet("/lakehouse/default/Files/new/week_31_data.parquet")
new_df.head()

StatementMeta(, 3ecf38ca-948d-4fd9-99ab-6cbf91e300e1, 83, Finished, Available, Finished)

,hvfhs_license_num,pickup_datetime,trip_miles,trip_time,base_passenger_fare,tolls,bcf,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag,cbd_congestion_fee,week_number
0,HV0003,2025-07-28 00:15:25,1.25,832,29.95,0.00,0.71,2.64,2.75,0.00,0.00,27.15,N,N,N,N,N,1.50,31
1,HV0003,2025-07-28 00:34:49,3.15,981,38.26,0.00,0.94,3.48,2.75,0.00,0.00,22.63,N,N,N,N,N,1.50,31
2,HV0003,2025-07-28 00:21:29,2.58,574,13.52,0.00,0.34,1.22,0.00,0.00,0.00,9.44,N,N,N,N,N,0.00,31
3,HV0003,2025-07-28 00:36:20,3.18,930,14.42,0.00,0.36,1.28,0.00,0.00,0.00,13.87,N,N,N,N,N,0.00,31
4,HV0003,2025-07-28 00:10:02,5.66,1740,51.58,0.00,1.29,4.58,2.75,0.00,0.00,26.48,N,N,N,N,N,0.00,31


In [82]:
# Shape 
print ("Shape of new data:" ,new_df.shape)

StatementMeta(, 3ecf38ca-948d-4fd9-99ab-6cbf91e300e1, 84, Finished, Available, Finished)

Shape of new data: (2547263, 19)


### Data Quality

In [83]:
# Describe 
new_df.describe()

StatementMeta(, 3ecf38ca-948d-4fd9-99ab-6cbf91e300e1, 85, Finished, Available, Finished)

,pickup_datetime,trip_miles,trip_time,base_passenger_fare,tolls,bcf,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,cbd_congestion_fee,week_number
count,2547263,2547263.00,2547263.00,2547263.00,2547263.00,2547263.00,2547263.00,2547263.00,2547263.00,2547263.00,2547263.00,2547263.00,2547263.00
mean,2025-07-30 03:28:43.935421952,4.97,1217.44,27.66,1.08,0.69,2.30,1.01,0.23,1.31,21.13,0.53,31.00
min,2025-07-28 00:00:00,0.00,1.00,-28.18,0.00,0.00,0.00,0.00,0.00,0.00,-4.68,0.00,31.00
25%,2025-07-29 05:56:52.500000,1.50,591.00,12.11,0.00,0.29,1.00,0.00,0.00,0.00,9.04,0.00,31.00
50%,2025-07-30 06:17:32,2.91,975.00,19.36,0.00,0.47,1.62,0.00,0.00,0.00,15.56,0.00,31.00
75%,2025-07-31 04:20:09,6.21,1568.00,33.04,0.00,0.82,2.82,2.75,0.00,0.00,26.73,1.50,31.00
max,2025-07-31 23:59:59,286.17,27572.00,1250.43,62.55,31.22,88.30,5.50,7.25,197.17,918.68,3.00,31.00
std,NaN,6.02,912.55,26.89,3.55,0.70,2.18,1.32,0.72,3.86,19.23,0.72,0.00


In [84]:
# Filter out 0, negative driver pay
new_df = new_df[new_df['driver_pay'] > 0]

# Filter out 0, negative base passenger fare
new_df = new_df[new_df['base_passenger_fare'] > 0]

StatementMeta(, 3ecf38ca-948d-4fd9-99ab-6cbf91e300e1, 86, Finished, Available, Finished)

In [85]:
# Nulls? 
new_df.isna().sum()

StatementMeta(, 3ecf38ca-948d-4fd9-99ab-6cbf91e300e1, 87, Finished, Available, Finished)

hvfhs_license_num       0
pickup_datetime         0
trip_miles              0
trip_time               0
base_passenger_fare     0
tolls                   0
bcf                     0
sales_tax               0
congestion_surcharge    0
airport_fee             0
tips                    0
driver_pay              0
shared_request_flag     0
shared_match_flag       0
access_a_ride_flag      0
wav_request_flag        0
wav_match_flag          0
cbd_congestion_fee      0
week_number             0
dtype: int64

In [86]:
# Info 
new_df.info()

StatementMeta(, 3ecf38ca-948d-4fd9-99ab-6cbf91e300e1, 88, Finished, Available, Finished)

<class 'pandas.core.frame.DataFrame'>
Index: 2546015 entries, 0 to 2547262
Data columns (total 19 columns):
 #   Column                Dtype         
---  ------                -----         
 0   hvfhs_license_num     object        
 1   pickup_datetime       datetime64[ns]
 2   trip_miles            float64       
 3   trip_time             int64         
 4   base_passenger_fare   float64       
 5   tolls                 float64       
 6   bcf                   float64       
 7   sales_tax             float64       
 8   congestion_surcharge  float64       
 9   airport_fee           float64       
 10  tips                  float64       
 11  driver_pay            float64       
 12  shared_request_flag   object        
 13  shared_match_flag     object        
 14  access_a_ride_flag    object        
 15  wav_request_flag      object        
 16  wav_match_flag        object        
 17  cbd_congestion_fee    float64       
 18  week_number           UInt32        
dtypes: UI

In [87]:
# Describe 
new_df.select_dtypes('float64').describe()

StatementMeta(, 3ecf38ca-948d-4fd9-99ab-6cbf91e300e1, 89, Finished, Available, Finished)

,trip_miles,base_passenger_fare,tolls,bcf,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,cbd_congestion_fee
count,2546015.00,2546015.00,2546015.00,2546015.00,2546015.00,2546015.00,2546015.00,2546015.00,2546015.00,2546015.00
mean,4.97,27.67,1.08,0.69,2.30,1.01,0.23,1.31,21.14,0.53
std,6.03,26.90,3.55,0.70,2.18,1.32,0.72,3.86,19.23,0.72
min,0.00,0.04,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00
25%,1.50,12.11,0.00,0.29,1.00,0.00,0.00,0.00,9.05,0.00
50%,2.91,19.37,0.00,0.47,1.62,0.00,0.00,0.00,15.57,0.00
75%,6.21,33.05,0.00,0.82,2.82,2.75,0.00,0.00,26.74,1.50
max,286.17,1250.43,62.55,31.22,88.30,5.50,7.25,197.17,918.68,3.00


### Derived Features

In [88]:
# Copy 
new_df1 = new_df.copy()

StatementMeta(, 3ecf38ca-948d-4fd9-99ab-6cbf91e300e1, 90, Finished, Available, Finished)

In [89]:
# Shape 
new_df1.shape

StatementMeta(, 3ecf38ca-948d-4fd9-99ab-6cbf91e300e1, 91, Finished, Available, Finished)

(2546015, 19)

In [90]:
# Time-based features
new_df1['pickup_hour'] = new_df1['pickup_datetime'].dt.hour
new_df1['pickup_dayofweek'] = new_df1['pickup_datetime'].dt.dayofweek
new_df1['is_weekend'] = new_df1['pickup_dayofweek'].isin([5, 6]).astype(int)
new_df1['pickup_dayofmonth'] = new_df1['pickup_datetime'].dt.day
# Time of day category (morning, afternoon, night, etc.)
new_df1['time_of_day'] = pd.cut(new_df1['pickup_hour'], bins=[0,6,12,18,24], 
        labels=['Night','Morning','Afternoon','Evening'], right=False)

StatementMeta(, 3ecf38ca-948d-4fd9-99ab-6cbf91e300e1, 92, Finished, Available, Finished)

In [91]:
# Columns
new_df1.columns

StatementMeta(, 3ecf38ca-948d-4fd9-99ab-6cbf91e300e1, 93, Finished, Available, Finished)

Index(['hvfhs_license_num', 'pickup_datetime', 'trip_miles', 'trip_time',
       'base_passenger_fare', 'tolls', 'bcf', 'sales_tax',
       'congestion_surcharge', 'airport_fee', 'tips', 'driver_pay',
       'shared_request_flag', 'shared_match_flag', 'access_a_ride_flag',
       'wav_request_flag', 'wav_match_flag', 'cbd_congestion_fee',
       'week_number', 'pickup_hour', 'pickup_dayofweek', 'is_weekend',
       'pickup_dayofmonth', 'time_of_day'],
      dtype='object')

In [92]:
# Drop columns 
new_df1.drop(['pickup_datetime', 'week_number', 'pickup_hour', 
                'pickup_dayofmonth'], axis=1, inplace=True)
new_df1.info()                

StatementMeta(, 3ecf38ca-948d-4fd9-99ab-6cbf91e300e1, 94, Finished, Available, Finished)

<class 'pandas.core.frame.DataFrame'>
Index: 2546015 entries, 0 to 2547262
Data columns (total 20 columns):
 #   Column                Dtype   
---  ------                -----   
 0   hvfhs_license_num     object  
 1   trip_miles            float64 
 2   trip_time             int64   
 3   base_passenger_fare   float64 
 4   tolls                 float64 
 5   bcf                   float64 
 6   sales_tax             float64 
 7   congestion_surcharge  float64 
 8   airport_fee           float64 
 9   tips                  float64 
 10  driver_pay            float64 
 11  shared_request_flag   object  
 12  shared_match_flag     object  
 13  access_a_ride_flag    object  
 14  wav_request_flag      object  
 15  wav_match_flag        object  
 16  cbd_congestion_fee    float64 
 17  pickup_dayofweek      int32   
 18  is_weekend            int64   
 19  time_of_day           category
dtypes: category(1), float64(10), int32(1), int64(2), object(6)
memory usage: 381.2+ MB


### 1/0 encoding

In [93]:
# Set license_num to category, trip time to float64 (same as other continuous variables)
new_df1['hvfhs_license_num'] = new_df1['hvfhs_license_num'].astype('category')
new_df1['trip_time'] = new_df1['trip_time'].astype('float64')

StatementMeta(, 3ecf38ca-948d-4fd9-99ab-6cbf91e300e1, 95, Finished, Available, Finished)

In [94]:
# Set features for encoding
encoded_features = new_df1.select_dtypes('object').columns 
encoded_features

StatementMeta(, 3ecf38ca-948d-4fd9-99ab-6cbf91e300e1, 96, Finished, Available, Finished)

Index(['shared_request_flag', 'shared_match_flag', 'access_a_ride_flag',
       'wav_request_flag', 'wav_match_flag'],
      dtype='object')

In [95]:
# Mapping features with Y/N to 1/0
mapping = {'Y': 1, 'N': 0}

# Loop through columns, map to new values from dic
for col in encoded_features: 
    new_df1[col] = new_df1[col].map(mapping)

StatementMeta(, 3ecf38ca-948d-4fd9-99ab-6cbf91e300e1, 97, Finished, Available, Finished)

In [96]:
# Info 
new_df1.info()

StatementMeta(, 3ecf38ca-948d-4fd9-99ab-6cbf91e300e1, 98, Finished, Available, Finished)

<class 'pandas.core.frame.DataFrame'>
Index: 2546015 entries, 0 to 2547262
Data columns (total 20 columns):
 #   Column                Dtype   
---  ------                -----   
 0   hvfhs_license_num     category
 1   trip_miles            float64 
 2   trip_time             float64 
 3   base_passenger_fare   float64 
 4   tolls                 float64 
 5   bcf                   float64 
 6   sales_tax             float64 
 7   congestion_surcharge  float64 
 8   airport_fee           float64 
 9   tips                  float64 
 10  driver_pay            float64 
 11  shared_request_flag   int64   
 12  shared_match_flag     int64   
 13  access_a_ride_flag    int64   
 14  wav_request_flag      int64   
 15  wav_match_flag        int64   
 16  cbd_congestion_fee    float64 
 17  pickup_dayofweek      int32   
 18  is_weekend            int64   
 19  time_of_day           category
dtypes: category(2), float64(11), int32(1), int64(6)
memory usage: 364.2 MB



### Predictions

#### Selecting Features (X,y)

In [100]:
# Columns 
new_df1.columns 

StatementMeta(, 3ecf38ca-948d-4fd9-99ab-6cbf91e300e1, 102, Finished, Available, Finished)

Index(['hvfhs_license_num', 'trip_miles', 'trip_time', 'base_passenger_fare',
       'tolls', 'bcf', 'sales_tax', 'congestion_surcharge', 'airport_fee',
       'tips', 'driver_pay', 'shared_request_flag', 'shared_match_flag',
       'access_a_ride_flag', 'wav_request_flag', 'wav_match_flag',
       'cbd_congestion_fee', 'pickup_dayofweek', 'is_weekend', 'time_of_day'],
      dtype='object')

In [101]:
# Target 
y = new_df1['driver_pay']

# Features 
X = new_df1.drop("driver_pay", axis=1)

StatementMeta(, 3ecf38ca-948d-4fd9-99ab-6cbf91e300e1, 103, Finished, Available, Finished)

In [102]:
# Shape
print ("Shape of features (X):", X.shape)
print ("Shape of target (y):", y.shape)

StatementMeta(, 3ecf38ca-948d-4fd9-99ab-6cbf91e300e1, 104, Finished, Available, Finished)

Shape of features (X): (2546015, 19)
Shape of target (y): (2546015,)


#### Loading Saved Model 
Tree-based

In [103]:
# Loading from Files/models
import joblib
loaded_model = joblib.load("/lakehouse/default/Files/models/LightGBM_model.pkl")

StatementMeta(, 3ecf38ca-948d-4fd9-99ab-6cbf91e300e1, 105, Finished, Available, Finished)

In [104]:
# Predictions on new data 
y_pred = loaded_model.predict(X)

StatementMeta(, 3ecf38ca-948d-4fd9-99ab-6cbf91e300e1, 106, Finished, Available, Finished)

#### Predictions Accuracy

In [105]:
# Accuracy 
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Mean Absolute Error (MAE)
mae = mean_absolute_error(y, y_pred)
# Mean Squared Error (MSE)
mse = mean_squared_error(y, y_pred)

# Root Mean Squared Error (RMSE)
rmse = np.sqrt(mse)

# R-squared (R²)
r2 = r2_score(y, y_pred)

# Print the results
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"R-squared (R²): {r2:.2f}")

StatementMeta(, 3ecf38ca-948d-4fd9-99ab-6cbf91e300e1, 107, Finished, Available, Finished)

Mean Absolute Error (MAE): 1.44
Mean Squared Error (MSE): 14.54
Root Mean Squared Error (RMSE): 3.81
R-squared (R²): 0.96


In [106]:
# Shape 
print ("The shape of features X:", X.shape)
print ("The shape of target y:", y.shape)
print ("The shape of predictions:", y_pred.shape)

StatementMeta(, 3ecf38ca-948d-4fd9-99ab-6cbf91e300e1, 108, Finished, Available, Finished)

The shape of features X: (2546015, 19)
The shape of target y: (2546015,)
The shape of predictions: (2546015,)


#### Re-building new data
With original features and predictions

In [107]:
# Actuals, predicted 
results_df =  pd.DataFrame({'actuals': y, "predicted": y_pred}, index=X.index)
results_df.head()

StatementMeta(, 3ecf38ca-948d-4fd9-99ab-6cbf91e300e1, 109, Finished, Available, Finished)

,actuals,predicted
0,27.15,16.34
1,22.63,20.66
2,9.44,9.82
3,13.87,14.30
4,26.48,30.24


In [108]:
# concat with initial training data
final_df = pd.concat ([new_df['pickup_datetime'] , new_df1, results_df], axis=1)
display(final_df)

StatementMeta(, 3ecf38ca-948d-4fd9-99ab-6cbf91e300e1, 110, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, cb214710-a833-4504-bfc9-a2673c79708c)

#### Save Predictions

In [28]:
# Save predictions to fabric delta table 
spark.createDataFrame(final_df).write.format("delta").mode(
    "overwrite").option("overwriteSchema", "true").saveAsTable("final_df_lgbm")

StatementMeta(, 41df73ba-acc2-4f9b-a183-52ca78bca51b, 30, Finished, Available, Finished)

In [109]:
# Append to fabric delta tables
spark.createDataFrame(final_df).write.format("delta") \
    .mode("append") \
    .saveAsTable("final_df_lgbm")

StatementMeta(, 3ecf38ca-948d-4fd9-99ab-6cbf91e300e1, 111, Finished, Available, Finished)